In [1]:
# 3rd order debug 
import tfim
import tfim_perturbation
import tfim_matrices
import numpy as np
from scipy import sparse
from scipy.sparse import linalg as spla
from scipy import linalg
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import progressbar
from scipy import optimize
import itertools

# Initial system specification
L = [5]
Jij_seed = 19
h_x_range = np.arange(0, 0.001, 0.00001)

PBC = True
J = 1

# Build lattice and basis
###################################
lattice = tfim.Lattice(L, PBC)
N = lattice.N
basis = tfim.IsingBasis(lattice)
###################################

# construct random J matrix
Jij = tfim.Jij_instance(N,J,"bimodal",Jij_seed) 
# Jij = np.array([[-1, 1, 1]])

# List out all the spin_states, corresponding indices and energies
Energies = -tfim.JZZ_SK_ME(basis,Jij)
for index in range(2**N):
    print(index, basis.state(index), Energies[index])
    
GS_energy, GS_indices = tfim_perturbation.GS(Energies)

100% (32 of 32) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


0 [0 0 0 0 0] -0.0
1 [0 0 0 0 1] -4.0
2 [0 0 0 1 0] 4.0
3 [0 0 0 1 1] -4.0
4 [0 0 1 0 0] -4.0
5 [0 0 1 0 1] -4.0
6 [0 0 1 1 0] 4.0
7 [0 0 1 1 1] -0.0
8 [0 1 0 0 0] 4.0
9 [0 1 0 0 1] 4.0
10 [0 1 0 1 0] 4.0
11 [0 1 0 1 1] -0.0
12 [0 1 1 0 0] -4.0
13 [0 1 1 0 1] -0.0
14 [0 1 1 1 0] -0.0
15 [0 1 1 1 1] -0.0
16 [1 0 0 0 0] -0.0
17 [1 0 0 0 1] -0.0
18 [1 0 0 1 0] -0.0
19 [1 0 0 1 1] -4.0
20 [1 0 1 0 0] -0.0
21 [1 0 1 0 1] 4.0
22 [1 0 1 1 0] 4.0
23 [1 0 1 1 1] 4.0
24 [1 1 0 0 0] -0.0
25 [1 1 0 0 1] 4.0
26 [1 1 0 1 0] -4.0
27 [1 1 0 1 1] -4.0
28 [1 1 1 0 0] -4.0
29 [1 1 1 0 1] 4.0
30 [1 1 1 1 0] -4.0
31 [1 1 1 1 1] -0.0


In [2]:
# Building blocks matrices
ES_1_indices = tfim_matrices.Hamming_set(basis, GS_indices, N, GS_indices)
PVP = tfim_matrices.PVP(basis, GS_indices, N)
PVQ1 = tfim_matrices.PVQ_1(basis, Jij, GS_indices, ES_1_indices, N, GS_energy)
Q1VP = np.transpose(PVQ1)
Q1VQ1 = tfim_matrices.Q_1VQ_1(basis, ES_1_indices, GS_indices, N)

# energy_gap_matrix_12 (EGM) denotes 1/(E_0-QH_0Q)^2 from Q1 to Q1
EGM_12 = tfim_matrices.energy_gap(basis, Jij, ES_1_indices, N, GS_energy, 2)
EGM_13 = tfim_matrices.energy_gap(basis, Jij, ES_1_indices, N, GS_energy, 3)
EGM_11 = tfim_matrices.energy_gap(basis, Jij, ES_1_indices, N, GS_energy, 1)

# Start building Hamiltonians
H_0 = tfim_perturbation.H_app_0(GS_energy, GS_indices)

H_app_1 = PVP

H_app_2 = PVQ1 @ EGM_11 @ Q1VP

H_app_3_exp = -0.5*(PVP @ PVQ1 @ EGM_12 @ Q1VP + np.transpose(PVP @ PVQ1 @ EGM_12 @ Q1VP)) + PVQ1 @ EGM_11 @ Q1VQ1 @ EGM_11 @ Q1VP

In [3]:
print(ES_1_indices)
print(GS_indices)

[ 0  2  6  7  8  9 10 11 13 14 17 18 20 21 22 23 24 25 29 31]
[ 1  3  4  5 12 19 26 27 28 30]


In [4]:
a = np.array([1,2,3,4,5])
print(np.argwhere(a == 5))

[[4]]


In [5]:
H_app_3 = np.zeros((len(GS_indices), len(GS_indices)))

# for column, GS_bra_1 in enumerate(GS_indices):
#     state_0 = basis.state(GS_bra_1)
#     for i in range(N):
#         basis.flip(state_0, i)
#         state_1_index = basis.index(state_0)
#         if state_1_index in GS_indices:
#             for j in range(N):
#                 basis.flip(state_0, j)
#                 state_2_index = basis.index(state_0)
#                 if state_2_index not in GS_indices:
#                     energy_gap = GS_energy - tfim_perturbation.state_energy(basis, Jij, state_2_index)
#                     for k in range(N):
#                         basis.flip(state_0, k)
#                         state_3_index = basis.index(state_0)
#                         GS_3_index = np.argwhere(np.array(GS_indices) == state_3_index)
#                         if len(GS_3_index) > 0:
#                             row = GS_3_index[0][0]
#                             H_app_3[row, column] += -0.5/(energy_gap**2)
#                         basis.flip(state_0, k)
#                 basis.flip(state_0, j)
#         basis.flip(state_0, i)

# term_2 = np.transpose(H_app_3)
# H_app_3 += term_2

#     for column, GS_bra_1 in enumerate(GS_indices):        
#         state_0 = basis.state(GS_bra_1)
#         for i in range(N):
#             basis.flip(state_0, i)
#             state_1_index = basis.index(state_0)
#             if state_1_index not in GS_indices:
#                 energy_gap = state_energy(basis, Jij, state_1_index) - GS_energy
#                 for j in range(N):
#                     basis.flip(state_0, j)
#                     state_2_index = basis.index(state_0)
#                     if state_2_index in GS_indices:
#                         for k in range(N):
#                             basis.flip(state_0, k)
#                             state_3_index = basis.index(state_0)
#                             GS_3_index = np.argwhere(np.array(GS_indices) == state_3_index)
#                             if len(GS_3_index) > 0:
#                                 row = GS_3_index[0][0]
#                                 H_app_3[row, column] += -0.5/(energy_gap**2)
#                             basis.flip(state_0, k)
#                     basis.flip(state_0, j)
#             basis.flip(state_0, i)

for column, GS_bra_1 in enumerate(GS_indices):
    state_0 = basis.state(GS_bra_1)
    for i in range(N):
        basis.flip(state_0, i)
        state_1_index = basis.index(state_0)
        if state_1_index not in GS_indices:
            energy_gap_1 = GS_energy - tfim_perturbation.state_energy(basis, Jij, state_1_index)
            for j in range(N):
                basis.flip(state_0, j)
                state_2_index = basis.index(state_0)
                if state_2_index not in GS_indices:
                    energy_gap_2 = GS_energy - tfim_perturbation.state_energy(basis, Jij, state_2_index)
                    for k in range(N):
                        basis.flip(state_0, k)
                        state_3_index = basis.index(state_0)
                        GS_3_index = np.argwhere(np.array(GS_indices) == state_3_index)
                        if len(GS_3_index) > 0:
                            row = GS_3_index[0][0]
                            H_app_3[row, column] += 1.0/(energy_gap_1*energy_gap_2)
                        basis.flip(state_0, k)
                basis.flip(state_0, j)
        basis.flip(state_0, i)
print(H_app_3-PVQ1 @ EGM_11 @ Q1VQ1 @ EGM_11 @ Q1VP)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [6]:
H_exp = tfim_perturbation.H_app_3rd(0.05, H_0, H_app_1, H_app_2, H_app_3_exp, J)

In [7]:
H_0 = tfim_perturbation.H_app_0(GS_energy, GS_indices)
V = tfim_perturbation.H_app_1(basis, GS_indices, N)
H_2 = tfim_perturbation.H_app_2(basis, Jij, GS_indices, N, GS_energy)
H_3 = tfim_perturbation.H_app_3(basis, Jij, GS_indices, N, GS_energy)
H_corr = tfim_perturbation.H_app_3rd(0.05, H_0, V, H_2, H_3, J)

In [8]:
H_exp - H_corr

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [9]:
# debug Q1VQ2 Matrix--this must be the bug for 4th order 
def Q_1VQ_2(basis, ES_1_indices, ES_2_indices, GS_indices, N):
    #ES_2_indices denotes the indices of all the states that are one Hamming distance away from ES_1_indices
    # QVQ matrix
    QVQ = np.zeros((len(ES_1_indices), len(ES_2_indices)))
    for column, ket in enumerate(ES_2_indices):
        state = basis.state(ket)
        for i in range(N):
            basis.flip(state, i)
            bra = basis.index(state)
            subspace_index = np.argwhere(ES_1_indices == bra)
            if len(subspace_index) > 0 and ES_1_indices[subspace_index] not in GS_indices:
                row = subspace_index[0][0]
                QVQ[row, column] += 1
            basis.flip(state,i)
    return QVQ

In [10]:
ES_1_indices = tfim_matrices.Hamming_set(basis, GS_indices, N, GS_indices)
ES_2_indices = tfim_matrices.Hamming_set(basis, ES_1_indices, N, GS_indices)

#ES_2_indices denotes the indices of all the states that are one Hamming distance away from ES_1_indices
# QVQ matrix
QVQ = np.zeros((len(ES_1_indices), len(ES_2_indices)))
for column, ket in enumerate(ES_2_indices):
    state = basis.state(ket)
    for i in range(N):
        basis.flip(state, i)
        bra = basis.index(state)
        subspace_index = np.argwhere(ES_1_indices == bra)
        if len(subspace_index) > 0 and ES_1_indices[subspace_index] not in GS_indices:
            row = subspace_index[0][0]
            QVQ[row, column] += 1
        basis.flip(state,i)
        
print(ES_2_indices.shape)
print(np.shape(QVQ))

(22,)
(20, 22)


In [14]:
app_eigenvalues = np.zeros((len(GS_indices), len(h_x_range)))
app_eigenstates = np.zeros((len(h_x_range), len(GS_indices), len(GS_indices)))

# Building blocks matrices
ES_1_indices = tfim_matrices.Hamming_set(basis, GS_indices, N, GS_indices)
ES_2_indices = tfim_matrices.Hamming_set(basis, ES_1_indices, N, GS_indices)

# Building blocks matrices
PVP = tfim_matrices.PVP(basis, GS_indices, N)
PVQ1 = tfim_matrices.PVQ_1(basis, Jij, GS_indices, ES_1_indices, N, GS_energy)
Q1VP = np.transpose(PVQ1)
Q1VQ1 = tfim_matrices.Q_1VQ_1(basis, ES_1_indices, GS_indices, N)
Q1VQ2 = tfim_matrices.Q_1VQ_2(basis, ES_2_indices, ES_1_indices, GS_indices, N)
Q2VQ1 = np.transpose(Q1VQ2)

# energy_gap_matrix_12 (EGM) denotes 1/(E_0-QH_0Q)^2 from Q1 to Q1
EGM_12 = tfim_matrices.energy_gap(basis, Jij, ES_1_indices, N, GS_energy, 2)
EGM_13 = tfim_matrices.energy_gap(basis, Jij, ES_1_indices, N, GS_energy, 3)
EGM_11 = tfim_matrices.energy_gap(basis, Jij, ES_1_indices, N, GS_energy, 1)
EGM_21 = tfim_matrices.energy_gap(basis, Jij, ES_2_indices, N, GS_energy, 1)

print(Q1VQ1.shape)
print(np.shape(Q1VQ2))

(20, 20)
(20, 22)


In [15]:
# Start building Hamiltonians
H_app_1 = PVP

H_app_2 = PVQ1 @ EGM_11 @ Q1VP

H_app_3 = -0.5*(PVP @ PVQ1 @ EGM_12 @ Q1VP + np.transpose(PVP @ PVQ1 @ EGM_12 @ Q1VP)) + PVQ1 @ EGM_11 @ Q1VQ1 @ EGM_11 @ Q1VP

H_app_4 = 0.5*(tfim_matrices.hc(PVQ1 @ EGM_13 @ Q1VP @ PVP @ PVP)) - 0.5*(tfim_matrices.hc(PVQ1 @ EGM_12 @ Q1VP @ PVQ1 @ EGM_11 @ Q1VP)) - 1.*(tfim_matrices.hc(PVQ1 @ EGM_11 @ Q1VQ1 @ EGM_12 @ Q1VP @ PVP)) + 1.*(PVQ1 @ EGM_11 @ Q1VQ2 @ EGM_21 @ Q2VQ1 @ EGM_11 @ Q1VP)

In [ ]:
print(0.5*(tfim_matrices.hc(PVQ1 @ EGM_13 @ Q1VP @ PVP @ PVP)))

In [ ]:
print(H_app_3)

In [ ]:
print(H_app_4)

In [ ]:
1/64